## QHDOPT for quadratic programming

In this notebook, we demonstrate how to employ QHDOPT to solve a quadratic programming problem with box constraints. 

Our target problem is $$\min \ f(x)=\frac{1}{2}x^TQx+b^Tx,$$ where $Q = \begin{bmatrix}-2 & 1 \\ 1 & -1 \end{bmatrix}, b = \begin{bmatrix}\frac{3}{4} \\ -\frac{1}{4}\end{bmatrix},$ and $x$ is a 2-dimensional variable vector with each entry constrained in $[0, 1]$.

We employ the QP mode of QHDOPT to input this problem.

### 1. Create problem instance

First, we import the class QHD from our package QHDOPT, implying the solver's algorithm. 

In [1]:
from qhdopt import QHD

ImportError: cannot import name 'BeforeValidator' from 'pydantic' (/Users/samuelkushnir/miniconda3/envs/QHDOPT-dev/lib/python3.9/site-packages/pydantic/__init__.cpython-39-darwin.so)

Next, we construct the matrices $Q$ and $b$ by Python lists. For the matrix $Q$, it is represented by a nested list. The vector $b$ is represented by a list, encoding its transposed matrix $b^T$. 

In [ ]:
Q = [[-2, 1],
     [1, -1]]
bt = [3/4, -1/4]

Then we create a problem instance, stored in a variable `model`. It mandates the matrices $Q$ and $b$ to construct the problem, and by default set the box constraints to the unit box $[0, 1]^n$. You may override the bounds by `bounds=(l, r)` or `bounds=[(l1, r1), ..., (ln, rn)]`. 

In [ ]:
model = QHD.QP(Q, bt)

### 2. Solve with D-Wave

Now we illustrate how to solve the problem with QHDOPT's solvers. We consider the D-Wave solver first. 

We can configure the D-Wave solver by running `model.dwave_setup` with all the parameters set. The mandatory parameter is the resolution $r$, which we set as 8. The API key can be either directly input by setting `api_key` or from a file. 

You may also set the annealing schedule, chain strength, embedding schemes, etc. Here we use default parameters. 

In [ ]:
model.dwave_setup(8, api_key_from_file='../dwave_API_key')

To compile, send the job to run, and post-processing, you can run `model.optimize`. Setting `verbose=1` outputs more runtime information.

In [5]:
model.optimize(verbose = 1)

Received Task from D-Wave:
2024-01-24 07:24:02
Backend QPU Time: 0.025398769999999998
Overhead Time: 4.9655310719952395
* Coarse solution
Minimizer: [0. 1.]
Minimum: -0.75

* Fine-tuned solution
Minimizer: [0. 1.]
Minimum: -0.75
Success rate: 0.49

* Runtime breakdown
SimuQ compilation: 0.000 s
Backend runtime: 4.991 s
Decoding time: 0.049 s
Fine-tuning time: 0.445 s
* Total time: 5.485 s


-0.75

Here, the coarse solution is one of the decoded solutions directly from D-Wave devices, fined-tuned solution is the best solution obtained by using classical local solvers to refine the coarse solutions, and the success rate is the portion of samples leading to the best solution. 

The D-Wave solver returns a global minimum at $x=\begin{bmatrix} 0 \\ 1 \end{bmatrix}$, with the minimum value $-0.75$. After fine-tuning, the minimum does not change in this case. 

A runtime breakdown is also provided to exhibit the time consumption of each step in the solver. 

### 2. Solve with QuTiP

The QHD algorithm can be deployed to different backends, thanks to SimuQ's Hamiltonian-based compilation scheme. Here we demonstrate how we can use a QuTiP-based solver to implement QHD and solve the QP problem. 

The workflow follows the same style. We first setup the QuTiP solver, then solve with `optimize`. 

In [6]:
model.qutip_setup(6, time_discretization=40)

In [7]:
model.optimize(verbose = 1)

Compiled.
Solved.
* Coarse solution
Minimizer: [0.16666667 1.        ]
Minimum: -0.4861111040744517

* Fine-tuned solution
Minimizer: [0. 1.]
Minimum: -0.75
Success rate: 0.64

* Runtime breakdown
SimuQ compilation: 4.065 s
Backend runtime: 44.638 s
Decoding time: 0.001 s
Fine-tuning time: 0.069 s
* Total time: 48.774 s


-0.75

### 3. Solve with IonQ

We can also solve the QP problem with IonQ backends. Similarly, we first setup the IonQ solver, then solve with `optimize`. 

In [8]:
model.ionq_setup(6, api_key_from_file='../ionq_API_key', time_discretization=10, shots = 1000, on_simulator=True)

In [9]:
model.optimize(verbose = 1)

{'id': 'a711150a-a932-42ad-bb6d-049d447b1421', 'status': 'ready', 'request': 1706081294}
* Coarse solution
Minimizer: [0.16666667 1.        ]
Minimum: -0.4861111040744517

* Fine-tuned solution
Minimizer: [0. 1.]
Minimum: -0.75
Success rate: 0.657

* Runtime breakdown
SimuQ compilation: 161.999 s
Backend runtime: 6.240 s
Decoding time: 0.054 s
Fine-tuning time: 2.914 s
* Total time: 171.206 s


-0.75